# Inference

## Dependencies and imports

In [2]:
import os

import time
import numpy as np
import pandas as pd
from tqdm import tqdm

import google.generativeai as genai

## Utility

In [4]:
PROMPT = "Sei un dipendente pubblico che deve scrivere dei documenti istituzionali italiani per renderli semplici e comprensibili per i cittadini. Ti verrà fornito un documento pubblico e il tuo compito sarà quello di riscriverlo applicando regole di semplificazione senza però modificare il significato del documento originale. Ad esempio potresti rendere le frasi più brevi, eliminare le perifrasi, esplicitare sempre il soggetto, utilizzare parole più semplicii, trasformare i verbi passivi in verbi di forma attiva, spostare le frasi parentetiche alla fine del periodo."

In [5]:
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

generation_config = {
  "candidate_count": 1,
  "temperature": 0.2,
  "top_p": 0.1,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE",
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE",
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE",
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE",
  },
]

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash-latest",
  safety_settings=safety_settings,
  generation_config=generation_config,
  system_instruction=PROMPT,
)

def predict(_texts_to_simplify):
  chat_session = model.start_chat(
    history=[]
  )

  outputs = []
  for _text_to_simplify in tqdm(_texts_to_simplify):
    try:
      response = chat_session.send_message(_text_to_simplify)
      outputs.append(response.candidates[0].content.parts[0].text)
      time.sleep(4)
    except Exception as e:
      print(_text_to_simplify, e)
      outputs.append("TODO: manual")
  return outputs

## Load datasets

In [6]:
df = pd.read_csv('./texts/original.csv',encoding='utf-8')

## Random predictions

In [7]:
for s in df.sample(1).to_dict(orient='records'):
  output = predict([s['original_text']])[0]
  print("\noriginal: ", s['original_text'])
  print("\nmodel: ", output)
  print('----------------')

100%|██████████| 1/1 [00:06<00:00,  6.61s/it]


original:  Mediante libero accesso agli uffici negli orari di apertura al pubblico si effettuano le seguenti prestazioni: 
- Informazioni generali; 
- Informazioni e chiarimenti relativi ad eventuali preavvisi di contravvenzione; 
- Ritiro e consegna atti

model:  Puoi venire negli uffici durante gli orari di apertura al pubblico per:

* Chiedere informazioni generali.
* Ricevere informazioni e chiarimenti sulle multe che ti sono state notificate.
* Ritirare e consegnare documenti. 

----------------


## Run all predictions

In [ ]:
df['simplified_text'] = predict(df['original_text'].tolist())

## Save simplified datasets

In [9]:
df.to_csv('./texts/gemini.csv', index=False)